# Money supply in several countries
Getting data for M0, M1, M2, M3

In [20]:
# standard imports
import numpy as np
import pandas as pd

## 1. The USA
Data available from the St.Louis Federal reserve API. 

The Federal Reserve discontinued the calculation of M3 from 2006.

* [M0](https://fred.stlouisfed.org/series/BOGMBASE) | BOGMBASE
* [M1](https://fred.stlouisfed.org/series/M1NS) | M1NS
* [M2](https://fred.stlouisfed.org/series/M2NS) | M2NS

In [21]:
from full_fred.fred_base import FredBase
from full_fred.fred import Fred
fredB = FredBase()
fred = Fred('../fred_api_key.txt')

In [22]:
fred

In [23]:
m1 = fred.get_series_df('M1NS')

In [24]:
fred_codes = {
    # dict to store the money supply type and its FRED series code 
    # type: code
    'M0': 'BOGMBASE',
    'M1': 'M1NS',
    'M2': 'M2NS'
    }

In [25]:
def get_us_money_supply(code_dict):
    # initiate df
    df = pd.DataFrame(columns=['date', 'value', 'type'])

    for type, code in code_dict.items():
        # drop useless columnms
        cur_df = fred.get_series_df(code).drop(['realtime_start', 'realtime_end'], axis=1)

        # set the type column
        cur_df['type'] = type

        # add to working df
        df = pd.concat([df, cur_df])

    # change value to number
    df['value'] = pd.to_numeric(df['value'])
    
    # M1 and M2 are given in billions, but M0 is given in millions
    # adjust M0 to billions
    df.loc[df['type'] == 'M0', 'value'] /= 1000

    # country column
    df['country'] = 'USA'

    df = df.sort_values(by=['date', 'type']) 
    df = df.reset_index(drop=True)

    return df


In [26]:
usa_money = get_us_money_supply(fred_codes)

In [28]:
usa_money

,date,value,type,country
0,1959-01-01,50.5,M0,USA
1,1959-01-01,142.2,M1,USA
2,1959-01-01,289.8,M2,USA
3,1959-02-01,49.8,M0,USA
4,1959-02-01,139.3,M1,USA
...,...,...,...,...
2305,2023-01-01,19601.6,M1,USA
2306,2023-01-01,21251.5,M2,USA
2307,2023-02-01,5321.0,M0,USA
2308,2023-02-01,19340.9,M1,USA


In [27]:
# check for nan values
usa_money[usa_money.isna().any(axis=1)]

,date,value,type,country


## 2. Canada
Data Available from the BOC's valet API - documentation [here](https://www.bankofcanada.ca/valet/docs#valet_api). I found the series ID's for the data that I needed by downloading the CSV file on [this page](https://www.bankofcanada.ca/rates/banking-and-financial-statistics/selected-monetary-aggregates-and-their-components-formerly-e1/).

I couldn't find a series for the monetary base so I ended up getting all the data from [Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1010011601). I used their tools to get the series I needed and downloaded it as a csv directly and getting rid of the unneeded stuff they add in manually. When I implement some automation, this will have to change, but it works for now.

In [2]:
can_money_input = pd.read_csv('./lake/money_supply_canada.csv')

In [29]:
can_money_input

,Date,M0,M1,M2,M3
0,January 1955,"2,227",..,..,..
1,February 1955,"2,214",..,..,..
2,March 1955,"2,219",..,..,..
3,April 1955,"2,221",..,..,..
4,May 1955,"2,232",..,..,..
...,...,...,...,...,...
811,August 2022,"310,021","1,580,825","2,358,110","3,340,092"
812,September 2022,"319,257","1,580,973","2,374,338","3,371,949"
813,October 2022,"309,182","1,566,375","2,376,916","3,377,242"
814,November 2022,"319,796","1,556,447","2,384,951","3,411,736"


In [30]:
def fix_can_money(df):
    df = df.copy()

    # 1) Date format
    # make the date a datetime column
    df['Date'] = pd.to_datetime(df['Date'])
    # year-month-day format
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    # 2) Wide format to long format for the values
    df = pd.melt(df, id_vars='Date', value_vars=['M0','M1','M2','M3'])

    # 3) Change column names
    df = df.rename(columns={'Date': 'date', 'variable': 'type'})

    # 4) Value column commas, missing values
    df['value'] = df['value'].str.replace(',','')
    df = df.replace('..', np.NAN)

    # 5) Convert value to billions
    df['value'] = pd.to_numeric(df['value'])
    df['value'] = df['value']/1000

    # 6) Add country column
    df['country'] = 'CAN'


    # 6) re-sort, reindex
    df = df.sort_values(by=['date', 'type']) 
    df = df.reset_index(drop=True)

    return df

In [31]:
can_money = fix_can_money(can_money_input)

In [32]:
can_money

,date,type,value,country
0,1955-01-01,M0,2.227,CAN
1,1955-01-01,M1,NaN,CAN
2,1955-01-01,M2,NaN,CAN
3,1955-01-01,M3,NaN,CAN
4,1955-02-01,M0,2.214,CAN
...,...,...,...,...
3259,2022-11-01,M3,3411.736,CAN
3260,2022-12-01,M0,317.937,CAN
3261,2022-12-01,M1,1549.669,CAN
3262,2022-12-01,M2,2393.203,CAN


# 3. The United Kingdom
Data from the [bank of England](https://www.bankofengland.co.uk/boeapps/database/default.asp).

No API. Downloaded a CSV direct from their website.

In [48]:
gbr_money_input = pd.read_csv('lake/bOfEng.csv')
gbr_money_input

,Date,M0,M4,M3,M1,M2
0,31 Mar 23,92097,NaN,NaN,NaN,NaN
1,28 Feb 23,91714,2253073.0,3584056.0,2368787.0,3067755.0
2,31 Jan 23,94162,2254378.0,3584887.0,2381519.0,3069370.0
3,31 Dec 22,95336,2288794.0,3588302.0,2385850.0,3069192.0
4,30 Nov 22,93892,2279783.0,3631858.0,2460870.0,3114931.0
...,...,...,...,...,...,...
641,31 Oct 69,3585,NaN,NaN,NaN,NaN
642,30 Sep 69,3565,NaN,NaN,NaN,NaN
643,31 Aug 69,3608,NaN,NaN,NaN,NaN
644,31 Jul 69,3603,NaN,NaN,NaN,NaN


In [64]:
def fix_gbr_money(df):
    df = df.copy()

    # 1) fix date
    df['Date'] = pd.to_datetime(df['Date'])
    df['date'] = df['Date'] + pd.Timedelta(days=1)
    df = df.drop('Date', axis=1)

    # 2) Wide format to long format for the values
    df = pd.melt(df, id_vars='date', value_vars=['M0','M1','M2','M3', 'M4'])

    # 3) Convert to billions
    df['value'] = df['value']/1000

    # 4) Add country Column
    df['country'] = 'GBR'

    # 5) Column name
    df = df.rename(columns={'variable': 'type'})

    # 6) rows for all the way to 2070 get added - delete them
    from datetime import datetime
    df = df[df['date'] < datetime.now()]

    return df

In [65]:
gbr_money = fix_gbr_money(gbr_money_input)

## Amalgamate 
Every country finished will be added to the dict and the corresponding country data will get added to the output file.

In [67]:
countries = {
    'USA': usa_money,
    'CAN': can_money,
    'GBR': gbr_money
}

In [72]:
def union_countries(country_dict):
    # union
    df = pd.concat([d for d in countries.values()])

    # fix date
    df['date'] = pd.to_datetime(df['date']).dt.date
    # df['date'] = df['date'].dt.date

    # re-sort, reindex
    df = df.sort_values(by=['date', 'type', 'country']) 
    df = df.reset_index(drop=True)

    return df

In [73]:
money_supply = union_countries(countries)

In [74]:
money_supply

,date,value,type,country
0,1955-01-01,2.227,M0,CAN
1,1955-01-01,NaN,M1,CAN
2,1955-01-01,NaN,M2,CAN
3,1955-01-01,NaN,M3,CAN
4,1955-02-01,2.214,M0,CAN
...,...,...,...,...
8584,2023-04-01,92.097,M0,GBR
8585,2023-04-01,NaN,M1,GBR
8586,2023-04-01,NaN,M2,GBR
8587,2023-04-01,NaN,M3,GBR


In [75]:
money_supply.to_csv('csv_files/money_supply.csv')